## Import

In [3]:
import pandas as pd

## smc_daliy_price data max date_gap
### Read the smc daily price data

In [15]:
adj_price = pd.read_csv('adj_smc_daily_price_no_0.csv', parse_dates=['date'])
adj_price.head()

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi
0,NaN,3IINFOTECH,NaN,2009-01-01,39.35,41.40,38.75,40.60,299.0,324.86,2373300.0
1,NaN,3IINFOTECH,NaN,2009-01-02,40.90,43.60,40.70,41.40,435.0,495.12,2567700.0
2,NaN,3IINFOTECH,NaN,2009-01-05,42.00,42.20,40.15,40.55,322.0,356.59,2578500.0
3,NaN,3IINFOTECH,NaN,2009-01-06,40.20,42.70,40.00,40.90,247.0,276.09,2675700.0
4,NaN,3IINFOTECH,NaN,2009-01-07,41.90,43.25,35.25,36.65,687.0,727.72,2459700.0


### Shift the date and calculate the date gap

In [16]:
adj_price = adj_price.sort_values(['nse_ticker', 'date'])
adj_price['next_date'] = adj_price.groupby('nse_ticker').date.shift(-1)
adj_price['date_gap'] = adj_price['next_date'] - adj_price['date']
adj_price['date_gap'] = adj_price['date_gap'].apply(lambda x: x.days)

In [17]:
adj_price.head()

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
0,NaN,3IINFOTECH,NaN,2009-01-01,39.35,41.40,38.75,40.60,299.0,324.86,2373300.0,2009-01-02,1.0
1,NaN,3IINFOTECH,NaN,2009-01-02,40.90,43.60,40.70,41.40,435.0,495.12,2567700.0,2009-01-05,3.0
2,NaN,3IINFOTECH,NaN,2009-01-05,42.00,42.20,40.15,40.55,322.0,356.59,2578500.0,2009-01-06,1.0
3,NaN,3IINFOTECH,NaN,2009-01-06,40.20,42.70,40.00,40.90,247.0,276.09,2675700.0,2009-01-07,1.0
4,NaN,3IINFOTECH,NaN,2009-01-07,41.90,43.25,35.25,36.65,687.0,727.72,2459700.0,2009-01-09,2.0


In [27]:
# check if any problematic ticker (date_gap is too big) exists 
adj_price.groupby(['nse_ticker'])['date_gap'].max().reset_index().sort_values('date_gap', ascending=False).reset_index(drop=True).head()

,nse_ticker,date_gap
0,TORNTPOWER,2654.0
1,NIITTECH,2654.0
2,SRF,2164.0
3,CHENNPETRO,2164.0
4,WOCKPHARMA,1982.0


In [28]:
max_gap = pd.pivot_table(adj_price, values='date_gap', index='nse_ticker', 
                         aggfunc=max).sort_values('date_gap', ascending=False).reset_index()
max_gap.head()

,nse_ticker,date_gap
0,TORNTPOWER,2654.0
1,NIITTECH,2654.0
2,SRF,2164.0
3,CHENNPETRO,2164.0
4,WOCKPHARMA,1982.0


In [29]:
# check how many tickers have a big gap in the price records
len(max_gap[max_gap['date_gap'] > 6])

52

In [8]:
# convert to a csv of the max gap_date based on each ticker
max_gap.to_csv('adj_smc_daily_price_max_gapdate.csv', index=False)

## healthcare price prediction data max date_gap

In [32]:
healthcare_prediction = pd.read_csv("auto_healthcare_dj_price_prediction.csv", parse_dates=['date'])
healthcare_prediction.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,APOLLOHOSP,1,-1,0.706357,0.706357
1,2019-05-21 00:00:00-04:00,BIOCON,-1,-1,-1.000000,-1.000000
2,2019-05-21 00:00:00-04:00,CADILAHC,-1,-1,-1.000000,-1.000000
3,2019-05-21 00:00:00-04:00,CIPLA,1,-1,0.871319,0.871319
4,2019-05-21 00:00:00-04:00,DRREDDY,1,1,0.550777,NaN


In [10]:
healthcare_prediction = healthcare_prediction.sort_values(['ticker', 'date'])
healthcare_prediction['next_date'] = healthcare_prediction.groupby('ticker')['date'].shift(-1)
healthcare_prediction['date_gap'] = healthcare_prediction['next_date'] - healthcare_prediction['date']
healthcare_prediction['date_gap'] = healthcare_prediction['date_gap'].apply(lambda x:x.days)

In [11]:
# healthcare_prediction['date_gap'].sort_values(ascending=False).head()

15492    1982.0
9996        6.0
9994        6.0
9991        6.0
9992        6.0
Name: date_gap, dtype: float64

In [12]:
# check the number of tickers
# healthcare_prediction.ticker.nunique()

10

In [13]:
healthcare_max_gap = healthcare_prediction.pivot_table(values='date_gap', index='ticker', 
                                                       aggfunc='max').sort_values('date_gap', ascending=False).reset_index()

In [14]:
# check if any problematic ticker exists in the healthcare price prediction data
healthcare_max_gap

,ticker,date_gap
0,WOCKPHARMA,1982.0
1,APOLLOHOSP,6.0
2,BIOCON,6.0
3,CIPLA,6.0
4,DRREDDY,6.0
5,GLENMARK,6.0
6,LUPIN,6.0
7,CADILAHC,5.0
8,PEL,5.0
9,TORNTPHARM,5.0


In [34]:
# the start-date of the prediction of the healthcare ticker
healthcare_prediction.groupby('ticker')['date'].min()

ticker
APOLLOHOSP    2014-05-14 00:00:00-04:00
BIOCON        2009-05-29 00:00:00-04:00
CADILAHC      2016-02-10 00:00:00-05:00
CIPLA         2009-05-25 00:00:00-04:00
DRREDDY       2009-05-25 00:00:00-04:00
GLENMARK      2014-02-12 00:00:00-05:00
LUPIN         2009-05-26 00:00:00-04:00
PEL           2017-05-15 00:00:00-04:00
TORNTPHARM    2016-05-18 00:00:00-04:00
WOCKPHARMA    2009-05-26 00:00:00-04:00
Name: date, dtype: object

In [16]:
healthcare = healthcare_prediction.ticker.unique()
sub_price_healthcare = adj_price[adj_price.nse_ticker.isin(healthcare)]

In [17]:
# the start-date of the price of the healthcare ticker
sub_price_healthcare.groupby('nse_ticker').date.min()

nse_ticker
APOLLOHOSP   2013-12-27
BIOCON       2009-01-01
CADILAHC     2015-09-28
CIPLA        2009-01-01
DRREDDY      2009-01-01
GLENMARK     2013-10-03
LUPIN        2009-01-01
PEL          2012-09-21
TORNTPHARM   2016-01-01
WOCKPHARMA   2009-01-01
Name: date, dtype: datetime64[ns]

In [18]:
# check if any problematic ticker exists in the healthcare price data
sub_price_healthcare.groupby("nse_ticker").date_gap.max().reset_index()

,nse_ticker,date_gap
0,APOLLOHOSP,6.0
1,BIOCON,6.0
2,CADILAHC,5.0
3,CIPLA,6.0
4,DRREDDY,6.0
5,GLENMARK,6.0
6,LUPIN,6.0
7,PEL,1555.0
8,TORNTPHARM,5.0
9,WOCKPHARMA,1982.0


In [19]:
# Query the gap date of the problematic ticker in the prediction data
healthcare_prediction.query("ticker == 'WOCKPHARMA'").sort_values('date_gap', ascending=False).head()

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
15492,2009-06-25 00:00:00-04:00,WOCKPHARMA,1,1,0.734567,NaN,2014-11-28 00:00:00-05:00,1982.0
9192,2015-04-01 00:00:00-04:00,WOCKPHARMA,1,1,0.714829,NaN,2015-04-06 00:00:00-04:00,5.0
7367,2016-04-13 00:00:00-04:00,WOCKPHARMA,1,-1,0.702100,NaN,2016-04-18 00:00:00-04:00,5.0
7471,2016-03-23 00:00:00-04:00,WOCKPHARMA,1,-1,0.567800,0.567800,2016-03-28 00:00:00-04:00,5.0
2816,2018-03-28 00:00:00-04:00,WOCKPHARMA,1,-1,0.652895,0.652895,2018-04-02 00:00:00-04:00,5.0


In [20]:
## Query the gap date of the problematic ticker in the price data
adj_price.query("nse_ticker == 'WOCKPHARMA'").sort_values('date_gap', ascending=False).head()

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
484554,WPL,WOCKPHARMA,BBG001SDN510,2009-06-25,144.00,148.5,142.50,148.00,179.0,626.86,1113600.0,2014-11-28,1982.0
484640,WPL,WOCKPHARMA,BBG001SDN510,2015-04-01,1861.10,1893.5,1856.30,1887.55,5236.0,49107.26,2631500.0,2015-04-06,5.0
485380,WPL,WOCKPHARMA,BBG001SDN510,2018-03-28,731.10,741.6,723.65,726.65,2659.0,17532.42,301500.0,2018-04-02,5.0
484895,WPL,WOCKPHARMA,BBG001SDN510,2016-04-13,1047.85,1058.0,1030.00,1035.50,5436.0,21361.81,2147625.0,2016-04-18,5.0
484515,WPL,WOCKPHARMA,BBG001SDN510,2009-04-29,91.40,95.4,88.15,88.75,337.0,746.43,290400.0,2009-05-04,5.0


## Auto energey price prediction data max date_gap

In [21]:
energy_prediction = pd.read_csv("auto_energy_dj_price_prediction.csv", parse_dates=['date'])
energy_prediction.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,BPCL,1,-1,0.900000,0.900000
1,2019-05-21 00:00:00-04:00,HINDPETRO,-1,-1,-0.594386,-0.594386
2,2019-05-21 00:00:00-04:00,IOC,1,-1,0.730989,NaN
3,2019-05-21 00:00:00-04:00,MRPL,-1,-1,-0.877875,NaN
4,2019-05-21 00:00:00-04:00,OIL,1,-1,0.632422,NaN


In [22]:
energy_prediction = energy_prediction.sort_values(['ticker', 'date'])
energy_prediction['next_date'] = energy_prediction.groupby('ticker')['date'].shift(-1)
energy_prediction['date_gap'] = energy_prediction['next_date'] - energy_prediction['date']
energy_prediction['date_gap'] = energy_prediction['date_gap'].apply(lambda x:x.days)

In [23]:
# energy_prediction['date_gap'].sort_values(ascending=False).head()

11291    1674.0
11292     974.0
8306        6.0
8303        6.0
8305        6.0
Name: date_gap, dtype: float64

In [24]:
# energy_prediction.ticker.nunique()

8

In [25]:
energy_max_gap = energy_prediction.pivot_table(values='date_gap', index='ticker', 
                                               aggfunc='max').sort_values('date_gap', ascending=False).reset_index()

In [26]:
energy_max_gap

,ticker,date_gap
0,MRPL,1674.0
1,OIL,974.0
2,BPCL,6.0
3,HINDPETRO,6.0
4,IOC,6.0
5,ONGC,6.0
6,PETRONET,6.0
7,RELIANCE,6.0


In [27]:
energy_prediction.groupby('ticker')['date'].min()

ticker
BPCL        2009-05-25 00:00:00-04:00
HINDPETRO   2009-05-26 00:00:00-04:00
IOC         2009-05-26 00:00:00-04:00
MRPL        2009-05-26 00:00:00-04:00
OIL         2011-03-16 00:00:00-04:00
ONGC        2009-05-25 00:00:00-04:00
PETRONET    2009-05-27 00:00:00-04:00
RELIANCE    2009-05-25 00:00:00-04:00
Name: date, dtype: datetime64[ns, tzoffset(None, -14400)]

In [28]:
energy = energy_prediction.ticker.unique()
sub_price_energy = adj_price[adj_price["nse_ticker"].isin(energy)]

In [29]:
sub_price_energy.groupby('nse_ticker').date.min()

nse_ticker
BPCL        2009-01-01
HINDPETRO   2009-01-01
IOC         2009-01-01
MRPL        2009-01-01
OIL         2010-10-29
ONGC        2009-01-01
PETRONET    2009-01-01
RELIANCE    2009-01-01
Name: date, dtype: datetime64[ns]

In [30]:
sub_price_energy.groupby("nse_ticker").date_gap.max().reset_index()

,nse_ticker,date_gap
0,BPCL,6.0
1,HINDPETRO,6.0
2,IOC,6.0
3,MRPL,1674.0
4,OIL,974.0
5,ONGC,6.0
6,PETRONET,6.0
7,RELIANCE,6.0


In [31]:
energy_prediction.query("ticker =='MRPL'").sort_values('date_gap', ascending=False).head()

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
11291,2012-09-27 00:00:00-04:00,MRPL,1,1,0.909219,0.909219,2017-04-28 00:00:00-04:00,1674.0
16483,2009-12-24 00:00:00-05:00,MRPL,1,1,0.704940,0.704940,2009-12-29 00:00:00-05:00,5.0
1321,2018-09-12 00:00:00-04:00,MRPL,1,-1,0.559267,NaN,2018-09-17 00:00:00-04:00,5.0
16189,2010-02-26 00:00:00-05:00,MRPL,-1,1,-0.960244,-0.960244,2010-03-03 00:00:00-05:00,5.0
12262,2012-04-04 00:00:00-04:00,MRPL,-1,1,-0.689684,-0.689684,2012-04-09 00:00:00-04:00,5.0


In [32]:
sub_price_energy.query("nse_ticker == 'MRPL'").sort_values('date_gap', ascending=False).head()

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
310178,MRPL,MRPL,BBG001S840W9,2012-09-27,59.25,60.65,58.10,60.10,214.0,509.08,616000.0,2017-04-28,1674.0
310407,MRPL,MRPL,BBG001S840W9,2018-03-28,111.45,111.45,108.20,109.75,993.0,4928.36,814500.0,2018-04-02,5.0
309487,MRPL,MRPL,BBG001S840W9,2009-12-24,78.00,79.20,77.75,78.35,564.0,1966.95,4605750.0,2009-12-29,5.0
309323,MRPL,MRPL,BBG001S840W9,2009-04-29,48.20,49.20,47.60,49.10,189.0,815.02,720900.0,2009-05-04,5.0
310056,MRPL,MRPL,BBG001S840W9,2012-04-04,68.25,68.70,67.45,68.40,43.0,117.14,1796000.0,2012-04-09,5.0


## Auto communication price prediction(both news and price) data max date_gap

In [33]:
communication_prediction = pd.read_csv("auto_communication_dj_price_prediction.csv", parse_dates=['date'])
communication_prediction.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,BHARTIARTL,1,-1,0.856928,0.856928
1,2019-05-21 00:00:00-04:00,IDEA,1,-1,0.587570,0.587570
2,2019-05-21 00:00:00-04:00,INFRATEL,1,1,0.522500,0.522500
3,2019-05-21 00:00:00-04:00,TATACOMM,-1,-1,-0.580018,NaN
4,2019-05-20 00:00:00-04:00,BHARTIARTL,1,1,0.902589,0.902589


In [34]:
communication_prediction = communication_prediction.sort_values(['ticker', 'date'])
communication_prediction['next_date'] = communication_prediction.groupby('ticker')['date'].shift(-1)
communication_prediction['date_gap'] = communication_prediction['next_date'] - communication_prediction['date']
communication_prediction['date_gap'] = communication_prediction['date_gap'].apply(lambda x:x.days)

In [35]:
# communication_prediction['date_gap'].sort_values(ascending=False).head()

5260    6.0
5262    6.0
3794    6.0
5259    6.0
5261    6.0
Name: date_gap, dtype: float64

In [36]:
# communication_prediction['ticker'].nunique()

5

In [37]:
communication_max_gap = communication_prediction.pivot_table(values='date_gap', index='ticker', 
                                                             aggfunc='max').sort_values('date_gap', ascending=False).reset_index()

In [38]:
communication_max_gap

,ticker,date_gap
0,BHARTIARTL,6.0
1,IDEA,6.0
2,RCOM,6.0
3,TATACOMM,6.0
4,INFRATEL,5.0


In [39]:
communication_prediction.groupby('ticker')['date'].min()

ticker
BHARTIARTL    2009-05-25 00:00:00-04:00
IDEA          2009-05-28 00:00:00-04:00
INFRATEL      2016-02-10 00:00:00-05:00
RCOM          2009-05-25 00:00:00-04:00
TATACOMM      2009-05-25 00:00:00-04:00
Name: date, dtype: object

In [40]:
communication = communication_prediction.ticker.unique()
sub_price_communication = adj_price[adj_price["nse_ticker"].isin(communication)]

In [63]:
sub_price_communication.groupby('nse_ticker').date.min().reset_index()

,nse_ticker,date
0,BHARTIARTL,2009-01-01
1,IDEA,2009-01-01
2,INFRATEL,2015-09-28
3,RCOM,2009-01-01
4,TATACOMM,2009-01-01


In [42]:
sub_price_communication.groupby("nse_ticker").date_gap.max().reset_index()

,nse_ticker,date_gap
0,BHARTIARTL,6.0
1,IDEA,6.0
2,INFRATEL,5.0
3,RCOM,6.0
4,TATACOMM,6.0


## Auto communication price prediction (price only) data max date_gap

In [43]:
communication_price_only = pd.read_csv("auto_communication_price_only_prediction.csv", parse_dates=['date'])
communication_price_only.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,BHARTIARTL,1,-1,0.800000,NaN
1,2019-05-21 00:00:00-04:00,IDEA,-1,-1,-0.878111,-0.878111
2,2019-05-21 00:00:00-04:00,INFRATEL,-1,1,-0.590387,-0.590387
3,2019-05-21 00:00:00-04:00,TATACOMM,-1,-1,-0.643513,NaN
4,2019-05-20 00:00:00-04:00,BHARTIARTL,-1,1,-0.600000,NaN


In [44]:
communication_price_only = communication_price_only.sort_values(['ticker', 'date'])
communication_price_only['next_date'] = communication_price_only.groupby('ticker')['date'].shift(-1)
communication_price_only['date_gap'] = communication_price_only['next_date'] - communication_price_only['date']
communication_price_only['date_gap'] = communication_price_only['date_gap'].apply(lambda x:x.days)

In [45]:
# communication_price_only['date_gap'].sort_values(ascending=False).head()

5260    6.0
5262    6.0
3794    6.0
5259    6.0
5261    6.0
Name: date_gap, dtype: float64

In [46]:
# communication_price_only['ticker'].nunique()

5

In [47]:
communication_price_only_max_gap = communication_price_only.pivot_table(values='date_gap', index='ticker', 
                                                                        aggfunc='max').sort_values('date_gap', ascending=False).reset_index()

In [48]:
communication_price_only_max_gap

,ticker,date_gap
0,BHARTIARTL,6.0
1,IDEA,6.0
2,RCOM,6.0
3,TATACOMM,6.0
4,INFRATEL,5.0


In [49]:
communication_price_only.groupby('ticker')['date'].min()

ticker
BHARTIARTL    2009-05-25 00:00:00-04:00
IDEA          2009-05-28 00:00:00-04:00
INFRATEL      2016-02-10 00:00:00-05:00
RCOM          2009-05-25 00:00:00-04:00
TATACOMM      2009-05-25 00:00:00-04:00
Name: date, dtype: object

In [50]:
communication_only = communication_price_only.ticker.unique()
sub_price_communication_only = adj_price[adj_price["nse_ticker"].isin(communication_only)]

In [51]:
sub_price_communication_only.groupby('nse_ticker').date.min()

nse_ticker
BHARTIARTL   2009-01-01
IDEA         2009-01-01
INFRATEL     2015-09-28
RCOM         2009-01-01
TATACOMM     2009-01-01
Name: date, dtype: datetime64[ns]

In [52]:
sub_price_communication_only.groupby("nse_ticker").date_gap.max().reset_index()

,nse_ticker,date_gap
0,BHARTIARTL,6.0
1,IDEA,6.0
2,INFRATEL,5.0
3,RCOM,6.0
4,TATACOMM,6.0


## Auto utilities price prediction data max date_gap

In [53]:
utilities_prediction = pd.read_csv('auto_utilities_price_only_prediction.csv', parse_dates=['date'])
utilities_prediction.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,CESC,-1,-1,-0.687360,-0.687360
1,2019-05-21 00:00:00-04:00,GAIL,1,-1,0.605790,NaN
2,2019-05-21 00:00:00-04:00,IGL,-1,-1,-0.678000,-0.678000
3,2019-05-21 00:00:00-04:00,NTPC,1,-1,0.535581,0.535581
4,2019-05-21 00:00:00-04:00,POWERGRID,-1,-1,-1.000000,-1.000000


In [54]:
utilities_prediction = utilities_prediction.sort_values(['ticker', 'date'])
utilities_prediction['next_date'] = utilities_prediction.groupby('ticker')['date'].shift(-1)
utilities_prediction['date_gap'] = utilities_prediction['next_date'] - utilities_prediction['date']
utilities_prediction['date_gap'] = utilities_prediction['date_gap'].apply(lambda x:x.days)

In [55]:
# utilities_prediction['date_gap'].sort_values(ascending=False).head()

9838     6.0
9839     6.0
9842     6.0
20645    6.0
9841     6.0
Name: date_gap, dtype: float64

In [56]:
# utilities_prediction.ticker.nunique()

10

In [57]:
utilities_max_gap = utilities_prediction.pivot_table(values='date_gap', index='ticker', 
                                                     aggfunc='max').sort_values('date_gap', ascending=False).reset_index()                                                                                            

In [58]:
utilities_max_gap

,ticker,date_gap
0,CESC,6.0
1,GAIL,6.0
2,IGL,6.0
3,JSWENERGY,6.0
4,NTPC,6.0
5,POWERGRID,6.0
6,RELINFRA,6.0
7,RPOWER,6.0
8,TATAPOWER,6.0
9,GSPL,5.0


In [59]:
utilities_prediction.groupby('ticker')['date'].min()

ticker
CESC         2009-05-25 00:00:00-04:00
GAIL         2009-05-25 00:00:00-04:00
GSPL         2009-05-28 00:00:00-04:00
IGL          2011-03-11 00:00:00-05:00
JSWENERGY    2011-12-01 00:00:00-05:00
NTPC         2009-05-26 00:00:00-04:00
POWERGRID    2009-05-27 00:00:00-04:00
RELINFRA     2009-05-25 00:00:00-04:00
RPOWER       2009-05-25 00:00:00-04:00
TATAPOWER    2009-05-25 00:00:00-04:00
Name: date, dtype: object

In [60]:
utilities =  utilities_prediction.ticker.unique()
sub_price_utilities = adj_price[adj_price["nse_ticker"].isin(utilities)]

In [61]:
sub_price_utilities.groupby('nse_ticker').date.min()

nse_ticker
CESC        2009-01-01
GAIL        2009-01-01
GSPL        2009-01-01
IGL         2010-10-29
JSWENERGY   2011-07-18
NTPC        2009-01-01
POWERGRID   2009-01-01
RELINFRA    2009-01-01
RPOWER      2009-01-01
TATAPOWER   2009-01-01
Name: date, dtype: datetime64[ns]

In [62]:
sub_price_utilities.groupby("nse_ticker").date_gap.max().reset_index()

,nse_ticker,date_gap
0,CESC,6.0
1,GAIL,6.0
2,GSPL,5.0
3,IGL,6.0
4,JSWENERGY,6.0
5,NTPC,6.0
6,POWERGRID,6.0
7,RELINFRA,6.0
8,RPOWER,6.0
9,TATAPOWER,6.0


# 7.10 Outer dj daily price direction prediction

In [35]:
import pandas as pd

In [38]:
daily_dir_pred = pd.read_csv("outer_dj_daily_oc_dir_prediction.csv", parse_dates=['date'])
daily_dir_pred.head()

,date,ticker,preds,truths,proba,proba_filtered
0,2019-05-21 00:00:00-04:00,ACEM,1,-1,0.506173,0.506173
1,2019-05-21 00:00:00-04:00,ADANI,-1,-1,-0.500000,NaN
2,2019-05-21 00:00:00-04:00,ADE,-1,1,-1.000000,-1.000000
3,2019-05-21 00:00:00-04:00,ADSEZ,1,-1,0.500014,NaN
4,2019-05-21 00:00:00-04:00,ALBK,1,-1,0.697494,0.697494


In [39]:
daily_dir_pred  = daily_dir_pred .sort_values(['ticker', 'date'])
daily_dir_pred['next_date'] = daily_dir_pred .groupby('ticker').date.shift(-1)
daily_dir_pred['date_gap'] = daily_dir_pred['next_date'].apply(lambda x:x.days) - daily_dir_pred['date'].apply(lambda x:x.days)
daily_dir_pred['date_gap'] = daily_dir_pred['date_gap'].apply(lambda x: x.days)

In [40]:
daily_dir_pred.head()

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
269674,2009-11-03 00:00:00-05:00,ACEM,-1,-1,-0.509639,NaN,2009-11-04 00:00:00-05:00,1.0
269597,2009-11-04 00:00:00-05:00,ACEM,1,1,0.501359,NaN,2009-11-05 00:00:00-05:00,1.0
269513,2009-11-05 00:00:00-05:00,ACEM,-1,-1,-0.621030,NaN,2009-11-06 00:00:00-05:00,1.0
269424,2009-11-06 00:00:00-05:00,ACEM,-1,-1,-0.805637,NaN,2009-11-09 00:00:00-05:00,3.0
269332,2009-11-09 00:00:00-05:00,ACEM,-1,-1,-0.817277,NaN,2009-11-10 00:00:00-05:00,1.0


In [43]:
# check if any problematic ticker (date_gap is too big) exists 
daily_dir = daily_dir_pred.groupby(['ticker'])['date_gap'].max().reset_index().sort_values('date_gap', 
                                                                                           ascending=False).reset_index(drop=True)

In [48]:
daily_dir.head(15)

,ticker,date_gap
0,NACL,1856.0
1,SREI,1737.0
2,FORH,1674.0
3,MRPL,1674.0
4,INBK,1646.0
5,CCRI,1646.0
6,SUEL,1491.0
7,RW,1310.0
8,KKC,1247.0
9,SINT,1037.0


In [54]:
daily_dir['ticker'].nunique()

163

In [120]:
gapticker = daily_dir[daily_dir['date_gap'] > 6].ticker
daily_dir[daily_dir['date_gap'] > 6]

,ticker,date_gap
0,NACL,1856.0
1,SREI,1737.0
2,FORH,1674.0
3,MRPL,1674.0
4,INBK,1646.0
5,CCRI,1646.0
6,SUEL,1491.0
7,RW,1310.0
8,KKC,1247.0
9,SINT,1037.0


In [112]:
gapticker

0     NACL
1     SREI
2     FORH
3     MRPL
4     INBK
5     CCRI
6     SUEL
7       RW
8      KKC
9     SINT
10    OINL
11     SIB
12     BOS
13    GMRI
14    TTMT
15      UT
16     JPA
Name: ticker, dtype: object

In [75]:
min_start_date = daily_dir_pred.groupby('ticker')['date'].min().reset_index()
min_start_date.head()

,ticker,date
0,ACEM,2009-11-03 00:00:00-05:00
1,ADANI,2011-05-23 00:00:00-04:00
2,ADE,2010-12-08 00:00:00-05:00
3,ADSEZ,2012-11-09 00:00:00-05:00
4,AL,2009-11-09 00:00:00-05:00


### NACL

In [69]:
daily_dir_pred.query("ticker == 'NACL'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
205065,2012-04-26 00:00:00-04:00,NACL,1,-1,0.999964,NaN,2017-05-26 00:00:00-04:00,1856.0
38848,2018-03-28 00:00:00-04:00,NACL,-1,-1,-0.999943,NaN,2018-04-02 00:00:00-04:00,5.0


In [76]:
min_start_date.query("ticker =='NACL'")

,ticker,date
98,NACL,2009-11-03 00:00:00-05:00


In [137]:
adj_price.query("bloomberg_ticker == 'NACL'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
314759,NACL,NATIONALUM,BBG001SBY0Y7,2012-04-26,60.65,60.65,57.1,57.85,128.0,301.80,376000.0,2017-05-26,1856.0
314745,NACL,NATIONALUM,BBG001SBY0Y7,2012-04-04,61.00,63.20,60.5,61.25,272.0,673.74,1388000.0,2012-04-09,5.0


In [138]:
min_price_start.query("bloomberg_ticker =='NACL'")

,bloomberg_ticker,date
97,NACL,2009-01-01


### SREI

In [78]:
daily_dir_pred.query("ticker == 'SREI'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
192060,2012-09-27 00:00:00-04:00,SREI,1,1,0.999999,NaN,2017-06-30 00:00:00-04:00,1737.0
47739,2017-12-21 00:00:00-05:00,SREI,1,1,0.578515,0.578515,2017-12-26 00:00:00-05:00,5.0


In [79]:
min_start_date.query("ticker =='SREI'")

,ticker,date
135,SREI,2011-08-23 00:00:00-04:00


In [139]:
adj_price.query("bloomberg_ticker == 'SREI'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
409999,SREI,SREINFRA,BBG001S84KM6,2012-09-27,25.70,26.55,25.45,26.35,185.0,384.54,1048000.0,2017-06-30,1737.0
410185,SREI,SREINFRA,BBG001S84KM6,2018-03-28,74.65,75.55,72.85,73.40,1782.0,6630.00,3340000.0,2018-04-02,5.0


In [140]:
min_price_start.query("bloomberg_ticker =='SREI'")

,bloomberg_ticker,date
133,SREI,2010-10-29


### FORH

In [81]:
daily_dir_pred.query("ticker == 'FORH'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
191989,2012-09-27 00:00:00-04:00,FORH,-1,1,-0.500000,-0.500000,2017-04-28 00:00:00-04:00,1674.0
206664,2012-04-04 00:00:00-04:00,FORH,1,1,0.579574,0.579574,2012-04-09 00:00:00-04:00,5.0


In [82]:
min_start_date.query("ticker =='FORH'")

,ticker,date
47,FORH,2010-12-08 00:00:00-05:00


In [142]:
adj_price.query("bloomberg_ticker == 'FORH'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
150599,FORH,FORTIS,BBG001SPGB72,2012-09-27,103.7,106.95,103.00,105.60,547.0,1143.85,1706000.0,2017-04-28,1674.0
150477,FORH,FORTIS,BBG001SPGB72,2012-04-04,99.5,102.90,99.05,102.25,803.0,1635.90,5036000.0,2012-04-09,5.0


In [143]:
min_price_start.query("bloomberg_ticker =='FORH'")

,bloomberg_ticker,date
46,FORH,2010-02-19


### MRPL

In [83]:
daily_dir_pred.query("ticker == 'MRPL'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
192029,2012-09-27 00:00:00-04:00,MRPL,-1,1,-0.526726,-0.526726,2017-04-28 00:00:00-04:00,1674.0
22773,2018-09-12 00:00:00-04:00,MRPL,-1,-1,-0.963872,-0.963872,2018-09-17 00:00:00-04:00,5.0


In [84]:
min_start_date.query("ticker =='MRPL'")

,ticker,date
93,MRPL,2009-11-04 00:00:00-05:00


In [144]:
adj_price.query("bloomberg_ticker == 'MRPL'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
310178,MRPL,MRPL,BBG001S840W9,2012-09-27,59.25,60.65,58.1,60.10,214.0,509.08,616000.0,2017-04-28,1674.0
310407,MRPL,MRPL,BBG001S840W9,2018-03-28,111.45,111.45,108.2,109.75,993.0,4928.36,814500.0,2018-04-02,5.0


In [145]:
min_price_start.query("bloomberg_ticker =='MRPL'")

,bloomberg_ticker,date
92,MRPL,2009-01-01


### INBK

In [85]:
daily_dir_pred.query("ticker == 'INBK'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
192012,2012-09-27 00:00:00-04:00,INBK,1,1,0.549720,0.54972,2017-03-31 00:00:00-04:00,1646.0
206686,2012-04-04 00:00:00-04:00,INBK,-1,-1,-0.567849,NaN,2012-04-09 00:00:00-04:00,5.0


In [86]:
min_start_date.query("ticker =='INBK'")

,ticker,date
74,INBK,2009-11-05 00:00:00-05:00


In [146]:
adj_price.query("bloomberg_ticker == 'INBK'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
229782,INBK,INDIANB,BBG001SKFSQ6,2012-09-27,186.25,187.75,183.30,186.55,198.0,367.82,93000.0,2017-03-31,1646.0
229660,INBK,INDIANB,BBG001SKFSQ6,2012-04-04,235.30,239.50,233.35,234.90,100.0,235.63,130000.0,2012-04-09,5.0


In [147]:
min_price_start.query("bloomberg_ticker =='INBK'")

,bloomberg_ticker,date
73,INBK,2009-01-01


### CCRI

In [95]:
daily_dir_pred.query("ticker == 'CCRI'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
228659,2011-06-30 00:00:00-04:00,CCRI,1,-1,0.63598,NaN,2016-01-01 00:00:00-05:00,1646.0
38791,2018-03-28 00:00:00-04:00,CCRI,-1,1,-1.00000,NaN,2018-04-02 00:00:00-04:00,5.0


In [96]:
min_start_date.query("ticker =='CCRI'")

,ticker,date
30,CCRI,2010-05-27 00:00:00-04:00


In [148]:
adj_price.query("bloomberg_ticker == 'CCRI'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
113645,CCRI,CONCOR,BBG001S86VR5,2011-06-30,229.152,231.466667,220.789333,222.197333,276.5625,155.09,63281.25,2016-01-01,1646.0
113715,CCRI,CONCOR,BBG001S86VR5,2016-04-13,408.176,412.000000,396.160000,406.000000,2081.2500,3363.87,1355000.00,2016-04-18,5.0


In [149]:
min_price_start.query("bloomberg_ticker =='CCRI'")

,bloomberg_ticker,date
30,CCRI,2009-01-01


### SUEL

In [153]:
daily_dir_pred.query("ticker == 'SUEL'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
180639,2013-02-28 00:00:00-05:00,SUEL,1,-1,1.0,1.0,2017-03-31 00:00:00-04:00,1491.0
69366,2017-04-27 00:00:00-04:00,SUEL,1,-1,1.0,1.0,2017-05-03 00:00:00-04:00,6.0


In [98]:
min_start_date.query("ticker =='SUEL'")

,ticker,date
138,SUEL,2009-11-04 00:00:00-05:00


In [152]:
adj_price.query("bloomberg_ticker == 'SUEL'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
423887,SUEL,SUZLON,BBG001SJDBD2,2013-02-28,24.65,25.5,14.20,15.90,15398.0,39453.18,86671000.0,2017-03-31,1492.0
423660,SUEL,SUZLON,BBG001SJDBD2,2012-04-04,25.10,25.7,24.55,24.85,4110.0,8213.24,103752000.0,2012-04-09,5.0


In [151]:
min_price_start.query("bloomberg_ticker =='SUEL'")

,bloomberg_ticker,date
136,SUEL,2009-01-01


### RW

In [99]:
daily_dir_pred.query("ticker == 'RW'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
165563,2013-09-26 00:00:00-04:00,RW,1,1,0.551701,NaN,2017-04-28 00:00:00-04:00,1310.0
185823,2012-12-21 00:00:00-05:00,RW,1,-1,0.999995,NaN,2012-12-26 00:00:00-05:00,5.0


In [100]:
min_start_date.query("ticker =='RW'")

,ticker,date
123,RW,2012-02-16 00:00:00-05:00


In [155]:
adj_price.query("bloomberg_ticker == 'RW'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
373616,RW,RAYMOND,BBG001S84H34,2013-09-26,208.60,223.75,203.0,220.25,2243.0,4807.30,625000.0,2017-04-28,1310.0
373845,RW,RAYMOND,BBG001S84H34,2018-03-28,911.75,926.35,894.0,897.30,2695.0,19620.83,369600.0,2018-04-02,5.0


In [156]:
min_price_start.query("bloomberg_ticker =='RW'")

,bloomberg_ticker,date
121,RW,2011-04-29


### KKC

In [101]:
daily_dir_pred.query("ticker == 'KKC'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
192022,2012-09-27 00:00:00-04:00,KKC,1,-1,1.000000,1.0,2016-02-26 00:00:00-05:00,1247.0
266284,2009-12-24 00:00:00-05:00,KKC,-1,-1,-0.727602,NaN,2009-12-29 00:00:00-05:00,5.0


In [102]:
min_start_date.query("ticker =='KKC'")

,ticker,date
84,KKC,2009-11-11 00:00:00-05:00


In [157]:
adj_price.query("bloomberg_ticker == 'KKC'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
118204,KKC,CUMMINSIND,BBG001S83W50,2012-09-27,505.65,509.0,481.00,487.45,752.0,1854.52,158500.0,2016-02-26,1247.0
118720,KKC,CUMMINSIND,BBG001S83W50,2018-03-28,717.30,725.5,683.55,700.20,2209.0,9344.69,256200.0,2018-04-02,5.0


In [158]:
min_price_start.query("bloomberg_ticker =='KKC'")

,bloomberg_ticker,date
83,KKC,2009-01-01


### SINT

In [103]:
daily_dir_pred.query("ticker == 'SINT'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
167547,2013-08-29 00:00:00-04:00,SINT,-1,-1,-0.599174,-0.599174,2016-07-01 00:00:00-04:00,1037.0
206727,2012-04-04 00:00:00-04:00,SINT,-1,-1,-0.500011,NaN,2012-04-09 00:00:00-04:00,5.0


In [104]:
min_start_date.query("ticker =='SINT'")

,ticker,date
130,SINT,2009-11-03 00:00:00-05:00


In [159]:
adj_price.query("bloomberg_ticker == 'SINT'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
406261,SINT,SINTEX,BBG001S82YH4,2013-08-29,20.3,20.3,17.9,18.30,3113.0,2315.29,5932000.0,2016-07-01,1037.0
405341,SINT,SINTEX,BBG001S82YH4,2009-12-24,126.9,130.0,125.6,128.65,1128.0,2025.58,1027600.0,2009-12-29,5.0


In [160]:
min_price_start.query("bloomberg_ticker =='SINT'")

,bloomberg_ticker,date
128,SINT,2009-01-01


### OINL

In [105]:
daily_dir_pred.query("ticker == 'OINL'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
192036,2012-09-27 00:00:00-04:00,OINL,1,1,0.693037,NaN,2015-05-29 00:00:00-04:00,974.0
206708,2012-04-04 00:00:00-04:00,OINL,1,1,0.654615,NaN,2012-04-09 00:00:00-04:00,5.0


In [106]:
min_start_date.query("ticker =='OINL'")

,ticker,date
105,OINL,2011-08-23 00:00:00-04:00


In [161]:
adj_price.query("bloomberg_ticker == 'OINL'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
337833,OINL,OIL,BBG001SQ2WX0,2012-09-27,236.625,240.00,236.05,239.400,130.0,155.17,101000.0,2015-05-29,974.0
338038,OINL,OIL,BBG001SQ2WX0,2016-03-23,159.850,159.85,157.65,158.675,556.0,1058.35,2709600.0,2016-03-28,5.0


In [162]:
min_price_start.query("bloomberg_ticker =='OINL'")

,bloomberg_ticker,date
104,OINL,2010-10-29


### SIB

In [107]:
daily_dir_pred.query("ticker == 'SIB'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
190062,2012-10-25 00:00:00-04:00,SIB,-1,-1,-1.000000,-1.0,2015-05-29 00:00:00-04:00,946.0
100589,2016-04-13 00:00:00-04:00,SIB,-1,1,-0.890473,NaN,2016-04-20 00:00:00-04:00,7.0


In [108]:
min_start_date.query("ticker =='SIB'")

,ticker,date
128,SIB,2012-05-21 00:00:00-04:00


In [163]:
adj_price.query("bloomberg_ticker == 'SIB'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
408535,SIB,SOUTHBANK,BBG001S6RGX2,2012-10-25,22.20,22.50,21.9,22.0,392.0,692.59,1664000.0,2015-05-29,946.0
408395,SIB,SOUTHBANK,BBG001S6RGX2,2012-04-04,24.85,25.05,24.8,25.0,42.0,104.63,5540000.0,2012-04-09,5.0


In [164]:
min_price_start.query("bloomberg_ticker =='SIB'")

,bloomberg_ticker,date
126,SIB,2011-07-18


### BOS

In [117]:
daily_dir_pred.query("ticker == 'BOS'").sort_values('date_gap', ascending=False).head(2)

,date,ticker,preds,truths,proba,proba_filtered,next_date,date_gap
191976,2012-09-27 00:00:00-04:00,BOS,-1,1,-0.592490,-0.592490,2014-11-28 00:00:00-05:00,792.0
193785,2012-09-06 00:00:00-04:00,BOS,1,-1,0.734322,0.734322,2012-09-13 00:00:00-04:00,7.0


In [118]:
min_start_date.query("ticker =='BOS'")

,ticker,date
24,BOS,2010-05-25 00:00:00-04:00


In [165]:
adj_price.query("bloomberg_ticker == 'BOS'").sort_values('date_gap', ascending=False).head(2)

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
78919,BOS,BOSCHLTD,BBG001S843M4,2012-09-27,8500.3,8670.0,8441.0,8526.25,33.0,351.51,2625.0,2014-11-28,792.0
78228,BOS,BOSCHLTD,BBG001S843M4,2009-12-24,4685.0,4685.0,4685.0,4685.00,0.0,0.00,2800.0,2009-12-29,5.0


In [166]:
min_price_start.query("bloomberg_ticker =='BOS'")

,bloomberg_ticker,date
24,BOS,2009-01-01


In [126]:
daily_pred_ticker =  daily_dir['ticker'].unique()
sub_daily_price = adj_price[adj_price["bloomberg_ticker"].isin(daily_pred_ticker)]

In [127]:
sub_daily_price.head()

,bloomberg_ticker,nse_ticker,entity_share_class_figi,date,adj_open,adj_high,adj_low,adj_close,adj_volume,value,adj_oi,next_date,date_gap
7651,ADE,ADANIENT,BBG001S8CL72,2010-02-19,500.0,503.9,493.00,496.55,142.0,283.06,28000.0,2010-02-22,3.0
7652,ADE,ADANIENT,BBG001S8CL72,2010-02-22,500.8,500.8,487.15,493.65,104.0,204.64,22400.0,2010-02-23,1.0
7653,ADE,ADANIENT,BBG001S8CL72,2010-02-23,490.0,493.8,489.05,492.25,40.0,78.78,24800.0,2010-02-24,1.0
7654,ADE,ADANIENT,BBG001S8CL72,2010-02-24,491.0,493.3,490.45,491.15,34.0,66.83,16800.0,2010-02-25,1.0
7655,ADE,ADANIENT,BBG001S8CL72,2010-02-25,492.0,492.0,483.00,485.05,49.0,95.47,5200.0,2010-02-26,1.0


In [133]:
min_price_start = sub_daily_price.groupby('bloomberg_ticker').date.min().reset_index()
min_price_start

160

In [141]:
price_sub = sub_daily_price.groupby("bloomberg_ticker").date_gap.max().reset_index().sort_values('date_gap', ascending=False).reset_index(drop=True)
price_sub.head(10)

,bloomberg_ticker,date_gap
0,SRF,2164.0
1,STR,1982.0
2,WPL,1982.0
3,MTCL,1903.0
4,NACL,1856.0
5,SREI,1737.0
6,FORH,1674.0
7,MRPL,1674.0
8,INBK,1646.0
9,CCRI,1646.0


In [136]:
price_sub[price_sub['bloomberg_ticker'].isin(gapticker)]

,bloomberg_ticker,date_gap
4,NACL,1856.0
5,SREI,1737.0
6,FORH,1674.0
7,MRPL,1674.0
8,INBK,1646.0
9,CCRI,1646.0
11,SUEL,1492.0
12,RW,1310.0
13,KKC,1247.0
14,SINT,1037.0
